<a href="https://colab.research.google.com/github/sujantkumarkv/extend_context/blob/main/extend_gemma_ctx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


## Install dependencies

In [ ]:
!pip install -q torch immutabledict sentencepiece # datasets

## Download model weights

In [ ]:
# Choose variant and machine type
VARIANT = '2b-it' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cuda' #@param ['cuda', 'cpu']

In [ ]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
# assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
# assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

## model implementation




In [ ]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [ ]:
import sys
sys.path.append('gemma_pytorch')

In [ ]:
from gemma_pytorch.gemma.config import get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

## Setup the model

In [ ]:
import torch

# Set up model config.
model_config = get_config_for_2b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

## Run inference

Below are examples for generating in chat mode and generating with multiple
requests.

The instruction-tuned Gemma models were trained with a specific formatter that
annotates instruction tuning examples with extra information, both during
training and inference. The annotations (1) indicate roles in a conversation,
and (2) delineate turns in a conversation. Below we show a sample code snippet
for formatting the model prompt using the user and model chat templates in a
multi-turn conversation. The relevant tokens are:

- `user`: user turn
- `model`: model turn
- `<start_of_turn>`: beginning of dialogue turn
- `<end_of_turn>`: end of dialogue turn

Read about the Gemma formatting for instruction tuning and system instructions
[here](https://ai.google.dev/gemma/docs/formatting).

In [ ]:
!wget https://raw.githubusercontent.com/sujantkumarkv/extend_context/main/rocstories.txt

--2024-06-17 15:07:28--  https://raw.githubusercontent.com/sujantkumarkv/extend_context/main/rocstories.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18007969 (17M) [text/plain]
Saving to: ‘rocstories.txt’

rocstories.txt      100%[===================>]  17.17M  --.-KB/s    in 0.1s    

2024-06-17 15:07:28 (163 MB/s) - ‘rocstories.txt’ saved [18007969/18007969]



In [ ]:
with open("rocstories.txt", 'r') as text_input:
    text = text_input.read()

In [ ]:
from gemma_pytorch.gemma.tokenizer import Tokenizer

tokenizer = Tokenizer(tokenizer_path)
# len(tokenizer.encode(text)) # 314936
tokens4k = tokenizer.encode(text)[: 4000]
tokens8k = tokenizer.encode(text)[: 8000]
tokens16k = tokenizer.encode(text)[: 16000]
# tokens32k = tokenizer.encode(text)[: 32768]

text4k = tokenizer.decode(tokens4k)
text8k = tokenizer.decode(tokens8k)
text16k = tokenizer.decode(tokens16k)

In [ ]:
len(text), len(tokenizer.encode(text)), len(tokens8k), len(tokens16k)

(18007898, 4086816, 8000, 16000)

In [ ]:
tokensprompt_4k = tokenizer.decode(tokens4k[3800: 3900])
tokensprompt_6k = tokenizer.decode(tokens8k[5800: 5900])
tokensprompt_8k = tokenizer.decode(tokens8k[7800: 7900])

print(f'tokensprompt_4k\n {tokensprompt_4k}\n')
print(f'tokensprompt_6k\n {tokensprompt_6k}')
print(f'tokensprompt_8k\n {tokensprompt_8k}')

tokensprompt_4k
 s other friends questioned it. Tom was slowly ditched by his friends. Eventually he had nobody to hang out with.
My sister used to paint a lot back in high school. She had painted one painting that was really good. She had submitted it to a local art gallery. After a few months of waiting, they accepted her piece. It was displayed in the gallery for several months.
Eva loved panda bears. When she heard her zoo was getting pandas, she was so excited. She

tokensprompt_6k
  so happy she didn't care if anyone made fun of her.
My cousin Natalia loves to write short stories. She is seven years old and quite imaginative. She wrote a short story called Alien. We are selling it as on AMAZON. Natalia's book has sold over a thousand copies.
Kelly was walking down the street. To her dismay she was shaken by a large mouse. The mouse was coming out of a trash can. Kelley was horrified. She immediately ran away.
Lou
tokensprompt_8k
 
Mr Blue is taking Mrs Blue out on a date. They de

In [ ]:
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt=f'Read the following properly and answer questions based on it. \n {text8k}'
    )
    + MODEL_CHAT_TEMPLATE.format(prompt='Yes, sure I can help with that.')
    + USER_CHAT_TEMPLATE.format(prompt='Natalia loves to write short stories. She is seven years old and quite imaginative. Write more on the story that Natalia wrote.')
    + '<start_of_turn>model\n'
)
# print('Chat prompt:\n', prompt)

model.generate(
    prompt,
    device=device,
    output_len=100,
)

'The went to make-and to make-he could have a few of the next time to make up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up up'

In [ ]:
|# Generate sample
model.generate(
    'Write a poem about an llm writing a poem.',
    device=device,
    output_len=60,
)

'\n\nA digital quill, its thoughts take flight,\nPenning verses, in the pale moonlight.\nLines churn and flow, a digital dance,\nA symphony of words, at the touch of the screen.\n\nThe words weave a tapestry,\nWith metaphors and similes they play.\n'

## Learn more

Now that you have learned how to use Gemma in Pytorch, you can explore the many
other things that Gemma can do in [ai.google.dev/gemma](https://ai.google.dev/gemma).
See also these other related resources:

- [Gemma model card](https://ai.google.dev/gemma/docs/model_card)
- [Gemma C++ Tutorial](https://ai.google.dev/gemma/docs/gemma_cpp)
- [Gemma formatting and system instructions](https://ai.google.dev/gemma/docs/formatting)